# 3.2

## Approach

We used the same functions that we created in the first part of the project but we did combine all those into one function called *gain*. This function returns a dictionary which includes two key-value pairs where the keys are **Entropy** and **InfoGain**. Both of these have a dictionary value which includes information on the entropy and information gain for the variables that gets passed while calling the function.

In [14]:
# Imporing the required library
import pandas as pd
import math

In [15]:
def gain(**kwargs):
    try:
        df = pd.read_csv(list(kwargs.values())[0], sep=" ")
    except TypeError:
        df = list(kwargs.values())[0]
    d = {}
    
    for i in df[list(kwargs.values())[1]].unique():
        d['Prob_'+list(kwargs.values())[1]+'_'+i] = df[df[list(kwargs.values())[1]] == i].shape[0]/df.shape[0]


    for i in df[list(kwargs.values())[1]].unique():
        for a in range(2,len(kwargs)):
            for j in df[list(kwargs.values())[a]].unique():
                d["prob_"+list(kwargs.values())[a]+'_'+j+"_"+i] = df[df[list(kwargs.values())[1]] == i][df[df[list(kwargs.values())[1]] == i][list(kwargs.values())[a]] == j].shape[0] / df[df[list(kwargs.values())[a]] == j].shape[0]
    

    entr = {}
    cls_ent = 0
    for i in df[list(kwargs.values())[1]].unique():
        cls_ent += (-d['Prob_'+list(kwargs.values())[1]+"_"+i])*math.log2(d['Prob_'+list(kwargs.values())[1]+"_"+i])
        
    entr["ent_"+list(kwargs.values())[1]] = cls_ent
    
    key_list = list(d.keys())
    for b in range(2,len(kwargs)):
        for j in df[list(kwargs.values())[b]].unique():
            ent = 0
            for k in key_list:
                if k.startswith('prob_'+list(kwargs.values())[b]+'_'+j):
                    try:
                        ent += (-d[k]) * math.log2(d[k])
                    except ValueError:
                        ent += 0
            entr["ent_"+list(kwargs.values())[b]+"_"+j] = ent


    infogain = {}
    key_list = list(entr.keys())
    for b in range(2,len(kwargs)):
        gain = entr['ent_'+list(kwargs.values())[1]]
        
        for k in key_list:
            if k.startswith("ent_"+list(kwargs.values())[b]):
                gain = gain - (entr[k] * (df[df[k.split("_")[1]] == k.split("_")[2]].shape[0]/df.shape[0]))
        
        infogain[list(kwargs.values())[b]] = round(gain,3)
    
    
    ent_infogain = {"Entropy": entr, "InfoGain": infogain}
    return ent_infogain

The *printer* function takes two arguements, one is the dataset and the other is the dictionary that is returned by the *gain* function. When we call the *gain* function, we get the desired output, but this *printer* function was created after seeing how Prof. Wolffe had displayed his output. This function tries to mimic what he had done but we did include our own representation of the split that happens after we determine which variable is best for the initial split looking at the information gain. We take the dictionary that is returned by *gain* and use the information from there in a bunch of print statements in an effort to display the output more clearly.

We take the variable with maximum value from the InfoGain dictionary and split on that variable. The for loop goes through the unique values of that variable and stores the index number of the instances that falls in that unique value as a list inside another list. So the first element in the main list is the list of instances that falls in the unique value. The second element of list is the mode of the response variable for the instances that falls in that unique value.

In [16]:
def printer(data, dic):
    data = pd.read_csv(data, sep = " ")
    
    print('{}: {}'.format("Training Examples", data.shape[0]))
    print()
    print('{}: {}'.format("System", round(list(dic["Entropy"].values())[0], 3)))
    print("--------------------------------------")
    print()
    
    
    var = []
    for i in range(1,len(list(dic["Entropy"]))):
        if list(dic["Entropy"])[i].split("_")[1] not in var:
            var.append(list(dic["Entropy"])[i].split("_")[1])
            p = list(dic["Entropy"])[i].split("_")[1]
            print (p)
            print()
            for j in range(len(list(dic["Entropy"]))):
                if list(dic["Entropy"])[j].split("_")[1] == p:
                    print ('{}: {}'.format(list(dic["Entropy"])[j].split("_")[2], round(list(dic["Entropy"].values())[j], 3)))

            for k in range(len(list(dic["InfoGain"]))):
                if list(dic["InfoGain"])[k] == p:
                    print ('{}: {}'.format("InfoGain", list(dic["InfoGain"].values())[k]))
            print("--------------------------------------")
            
    
    var = max(dic["InfoGain"], key = dic["InfoGain"].get)
    dic1 = {}
    print("Splitting on", var)
    for i in range(len(data[var].unique())):
        dic1[data[var].unique()[i]] = [(data[data[var] == data[var].unique()[i]].index+1).tolist(), data[data[var] == data[var].unique()[i]].Oracle.mode()[0]]
    return dic1

In [17]:
treatment = gain(data= "treatment.data", dep= "Oracle", i1= "Pulse", i2= "BP", i3 = "Age")

In [18]:
printer("treatment.data", treatment)

Training Examples: 5

System: 0.971
--------------------------------------

Pulse

Strong: 0.918
Weak: 1.0
InfoGain: 0.02
--------------------------------------
BP

Normal: 0.0
Abnormal: 0.0
InfoGain: 0.971
--------------------------------------
Age

Teen: 0.0
Adult: 0.0
Senior: 0.918
InfoGain: 0.42
--------------------------------------
Splitting on BP


{'Normal': [[1, 2, 3], 'Pos'], 'Abnormal': [[4, 5], 'Neg']}

----------

In [19]:
fishing = gain(data= "fishing(1).data", dep= "Oracle", i1= "Wind", i2= "Air", i3 = "Water", i4 = "Sky")

In [20]:
printer("fishing(1).data", fishing)

Training Examples: 14

System: 0.985
--------------------------------------

Wind

Strong: 0.811
Weak: 0.918
InfoGain: 0.128
--------------------------------------
Air

Hot: 0.863
Cold: 0.985
InfoGain: 0.061
--------------------------------------
Water

Warm: 0.811
Moderate: 0.918
Cool: 0.811
InfoGain: 0.128
--------------------------------------
Sky

Sunny: 0.811
Cloudy: 0.0
Rainy: 0.722
InfoGain: 0.264
--------------------------------------
Splitting on Sky


{'Sunny': [[1, 2, 7, 8, 9, 11, 12, 13], 'Yes'],
 'Cloudy': [[3], 'Yes'],
 'Rainy': [[4, 5, 6, 10, 14], 'No']}

------------

In [21]:
donors = gain(data= "donors.data", dep= "Oracle", i1= "Status", i2= "Party")

In [22]:
printer("donors.data", donors)

Training Examples: 20

System: 0.881
--------------------------------------

Status

Single: 0.0
Married: 0.722
Family: 0.985
InfoGain: 0.175
--------------------------------------
Party

Democrat: 0.845
Republican: 0.918
InfoGain: 0.003
--------------------------------------
Splitting on Status


{'Single': [[1, 11, 15], 'No'],
 'Married': [[2, 4, 5, 7, 8, 12, 13, 17, 18, 19], 'No'],
 'Family': [[3, 6, 9, 10, 14, 16, 20], 'Yes']}

## Conclusion

Most of the work was completed in the first part of the project where we had to calculate the information gain. Because of this reason, part 2 of the project was very straightforward which only included the task of splitting the variable and making a decision based on the maximum occurance of instances.